
# Emotion Detection Model Analysis

This notebook provides a comprehensive analysis of the Emotion Detection Model. The analysis includes loading a pre-trained model, processing video data frame by frame to predict emotions, and saving the results which consist of emotion and confidence level per frame.

## Table of Contents
1. Load Pre-trained Model
2. Video Processing and Emotion Prediction
3. Saving the Results
    

In [1]:
from keras.models import  load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

## Load the model

run 
```python
    model.summary()
```
to see the model architecture.


In [ ]:
model_name = 'Riley_12' #This is the name of the model you want to load, other option is 'Riley_4'
model = load_model(f'{model_name}.h5', compile=False)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'Riley_12.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

: 

In [6]:
class_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [2]:
def preprocess_image(frame, target_size):
    img = cv2.resize(frame, target_size)  # Resize to 64x64 pixels
    img = img / 255.0  # Normalize pixel values to [0, 1]
    img = np.expand_dims(img, axis=-1)  # Add channel dimension for grayscale
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to predict emotion for a frame
def predict_emotion(frame):
    preprocessed_frame = preprocess_image(frame, (48, 48))
    prediction = model.predict(preprocessed_frame)
    predicted_class_index = np.argmax(prediction)
    predicted_class = class_labels[predicted_class_index]
    confidence_level = prediction[0][predicted_class_index]
    return predicted_class, confidence_level, prediction

In [8]:
person = 'person3' #This is the name of the person you want to test, other options are 'person1' and 'person2'
video_path = f"IMG_4296.MOV" #This is the path to the video you want to test
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error opening video stream or file")


## Video Processing and Emotion Prediction

Here, we import the video and process it frame by frame. For each frame, the model predicts the emotion displayed on the face. The predicted emotion and its confidence level are stored for further analysis.
            

In [9]:
frame_interval = 6

data = pd.DataFrame(columns=['frame', 'emotion', 'confidence'])
predictions = []
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Process every n-th frame based on the frame_interval
    if frame_count % frame_interval == 0:
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        predicted_class, confidence_level, prediction = predict_emotion(gray_frame)
        data = pd.concat([data, pd.DataFrame({'frame': frame_count, 'emotion': predicted_class, 'confidence': confidence_level}, index=[0])], ignore_index=True)
        predictions.append(predictions)
        print(f'Frame {frame_count}: {predicted_class} ({confidence_level:.2f})')

    frame_count += 1

2025-01-23 23:29:26.587428: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-01-23 23:29:26.601070: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Frame 0: sad (0.29)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Frame 6: happy (0.27)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Frame 12: happy (0.24)


/var/folders/4p/0l9p6q8966q_mfyj8tsjtww00000gn/T/ipykernel_14623/386822539.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, pd.DataFrame({'frame': frame_count, 'emotion': predicted_class, 'confidence': confidence_level}, index=[0])], ignore_index=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Frame 18: sad (0.24)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Frame 24: happy (0.25)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Frame 30: sad (0.22)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Frame 36: angry (0.21)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Frame 42: sad (0.26)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Frame 48: fear (0.23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Frame 54: happy (0.25)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Frame 60: happy (0.28)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Frame 66: happy (0.29)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Frame 72: sad (0.23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Frame 78: happy (0.22)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Frame 84: sad (0.23)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Frame 90: happy (0.26)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Frame 96: happy (0.26)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Frame 102: fear (0.24)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Frame 108: sad (0.26)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/ste

In [10]:
data.to_csv(f'outputdata/{person}_{model_name}.csv', index=False)

In [16]:
# Face detection cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Emotion labels
class_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

def preprocess_frame(frame, face_rect):
    x, y, w, h = face_rect
    # Extract and process face region
    face = frame[y:y+h, x:x+w]
    # Convert to grayscale
    gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    # Resize to 48x48
    resized = cv2.resize(gray, (48, 48))
    # Normalize
    normalized = resized / 255.0
    # Reshape for model
    processed = normalized.reshape(1, 48, 48, 1)
    return processed

# Load model
model = load_model('Riley_12.h5')

# Setup camera
cap = cv2.VideoCapture(0)

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Detect faces
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        for (x, y, w, h) in faces:
            # Process face and predict
            face_rect = (x, y, w, h)
            input_face = preprocess_frame(frame, face_rect)
            predictions = model.predict(input_face, verbose=0)[0]
            
            # Get prediction and confidence
            pred_idx = np.argmax(predictions)
            confidence = predictions[pred_idx] * 100
            emotion = class_labels[pred_idx]
            
            # Draw rectangle and label
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            label = f"{emotion}: {confidence:.1f}%"
            cv2.putText(frame, label, (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        # Show frame
        cv2.imshow('Camera Feed', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 